In [0]:
CREATE SCHEMA training.bronze;
CREATE SCHEMA training.silver;
CREATE SCHEMA training.gold;

In [0]:
-- Create bronze tables from raw CSV files
CREATE OR REPLACE TABLE training.bronze.customers
AS SELECT * FROM csv.`/Volumes/training/default/landing/customer.csv` WITH (header="true");

CREATE OR REPLACE TABLE training.bronze.products
AS SELECT * FROM csv.`/Volumes/training/default/landing/product.csv` WITH (header="true");

CREATE OR REPLACE TABLE training.bronze.orders
AS SELECT * FROM csv.`/Volumes/training/default/landing/order.csv` WITH (header="true");

In [0]:
-- Clean customers: Standardize names and emails
CREATE OR REPLACE TABLE training.silver.customers
AS SELECT
  customer_id,
  initcap(first_name) AS first_name,
  initcap(last_name) AS last_name,
  lower(email) AS email,
  initcap(city) AS city
FROM training.bronze.customers;

-- Clean products: Handle missing values and data types
CREATE OR REPLACE TABLE training.silver.products
AS SELECT
  product_id,
  initcap(product_name) AS product_name,
  initcap(category) AS category,
  CAST(price AS DECIMAL(10,2)) AS price
FROM training.bronze.products
WHERE price IS NOT NULL;

-- Enrich orders: Add calculated columns and validation
CREATE OR REPLACE TABLE training.silver.orders
AS SELECT
  o.order_id,
  o.customer_id,
  o.product_id,
  CAST(o.order_date AS DATE) AS order_date,
  CAST(o.quantity AS INT) AS quantity,
  CAST(p.price AS DECIMAL(10, 2)) AS price,
  (CAST(o.quantity AS INT) * CAST(p.price AS DECIMAL(10, 2))) AS order_total,
  CASE WHEN CAST(o.quantity AS INT) > 0 THEN 'Valid' ELSE 'Invalid' END AS status
FROM training.bronze.orders o
JOIN training.bronze.products p 
  ON o.product_id = p.product_id;


In [0]:
-- Customer Lifetime Value (CLV)
CREATE OR REPLACE TABLE training.gold.customer_clv
AS SELECT
  c.customer_id,
  c.first_name,
  c.last_name,
  c.city,
  SUM(o.order_total) AS total_spend,
  COUNT(o.order_id) AS order_count,
  AVG(o.order_total) AS avg_order_value
FROM training.silver.customers c
JOIN training.silver.orders o 
  ON c.customer_id = o.customer_id
WHERE o.status = 'Valid'
GROUP BY c.customer_id, c.first_name, c.last_name, c.city;

-- Product Performance
CREATE OR REPLACE TABLE training.gold.product_performance
AS SELECT
  p.product_id,
  p.product_name,
  p.category,
  SUM(o.quantity) AS total_units_sold,
  SUM(o.order_total) AS total_revenue,
  AVG(p.price) AS avg_price
FROM training.silver.products p
JOIN training.silver.orders o 
  ON p.product_id = o.product_id
WHERE o.status = 'Valid'
GROUP BY p.product_id, p.product_name, p.category;

-- Daily Sales Summary
CREATE OR REPLACE TABLE training.gold.daily_sales
AS SELECT
  order_date,
  SUM(order_total) AS daily_revenue,
  COUNT(order_id) AS daily_orders,
  SUM(quantity) AS daily_units
FROM training.silver.orders
WHERE status = 'Valid'
GROUP BY order_date;


In [0]:
-- Check row counts
SELECT 
  (SELECT COUNT(*) FROM training.bronze.customers) AS bronze_customers,
  (SELECT COUNT(*) FROM training.silver.customers) AS silver_customers,
  (SELECT COUNT(*) FROM training.gold.customer_clv) AS gold_customers;

In [0]:
-- Sample gold data
SELECT * FROM training.gold.product_performance 
ORDER BY total_revenue DESC 
LIMIT 5;
